In [26]:
import os
import sys
# sys.path.append("/home/ziyizhang/Desktop/Projects/mitsuba-uni/mitsuba3/build/python")
sys.path.insert(0, "/home/ziyizhang/Desktop/Projects/mitsuba-curve/mitsuba3/build/python")

import mitsuba as mi
import drjit as dr

from typing import Union
import matplotlib.pyplot as plt
import numpy as np


print("Loaded Mitsuba from: ", os.path.dirname(mi.__file__))
print("Loaded Mitsuba from: ", os.path.dirname(dr.__file__))

mi.set_variant("cuda_ad_rgb")
# mi.set_variant("scalar_rgb")

# dr.set_log_level(dr.LogLevel.Trace)
# mi.set_log_level(mi.LogLevel.Trace)

# scene_path = "/home/qiyuan/sp/mitsuba3/notebooks/bspline_curve.xml"
# scene = mi.load_file("cornell_curve.xml")

Loaded Mitsuba from:  /home/ziyizhang/Desktop/Projects/mitsuba-curve/mitsuba3/build/python/mitsuba
Loaded Mitsuba from:  /home/ziyizhang/Desktop/Projects/mitsuba-curve/mitsuba3/build/python/drjit


In [27]:
T = mi.ScalarTransform4f
scene_dict = mi.cornell_box()
scene_dict["curve1"] = {
    "type": "bspline",
    "filename": "data_samples/curve1.txt",
    "to_world": T.translate([0, 0.2, 0.2]) @ T.rotate([1, 0, 0], -40) @ T.scale(0.8),
    "bsdf": {
        "type": "ref",
        "id": "white"
    }
}
scene = mi.load_dict(scene_dict)

In [28]:
bsp = scene.shapes()[2]
params = mi.traverse(bsp)
params

SceneParameters[
  -----------------------------------------------------------------------------
  Name                      Flags    Type            Parent
  -----------------------------------------------------------------------------
  control_point_count                int             BSpline
  vertex                             Float           BSpline
  radius                             Float           BSpline
  bsdf.reflectance.value    ∂        Color3f         SRGBReflectanceSpectrum
]

In [29]:
image = mi.render(scene, spp=1024)
mi.Bitmap(image)

Bitmap[
  pixel_format = rgb,
  component_format = float32,
  size = [256, 256],
  srgb_gamma = 0,
  struct = Struct<12>[
    float32 R; // @0, premultiplied alpha
    float32 G; // @4, premultiplied alpha
    float32 B; // @8, premultiplied alpha
  ],
  data = [ 768 KiB of image data ]
]

In [24]:
t = dr.linspace(mi.Float, 0, dr.pi * 12, 300)
t_max = dr.max(t)
x = dr.cos(t) * t / t_max
y = dr.sin(t) * t / t_max
z = t * 0
r = 0.03 + dr.sin(t / 3.0) * 0.02
pt = mi.Point3f(x, y, z)
np.savetxt("./data_samples/curve1.txt", np.array([x.numpy(), y.numpy(), z.numpy(), r.numpy()]).T)

In [25]:
import meshplot as mp
mp.plot(pt.numpy(), shading={"point_size": 0.1})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0418312…